# Script to analyze GitHub trend data and upload results to Dynamo

In [32]:
# VC Sourcing Pipeline - Complete Setup and Configuration
import aiohttp  # type: ignore
import asyncio
import json
import os
import traceback
from typing import Dict, List, Optional, Any
from dotenv import load_dotenv  # type: ignore
from openai import AsyncOpenAI, ChatCompletion  # type: ignore
from pydantic import BaseModel  # type: ignore
from exa_py import Exa  # type: ignore
from github import Github  # type: ignore

# Load environment variables
load_dotenv()

# API Configuration
BASE_OSS_INSIGHT_URL = "https://api.ossinsight.io/v1"
TRENDING_REPOS_URL = f"{BASE_OSS_INSIGHT_URL}/trends/repos"
COLLECTIONS_URL = f"{BASE_OSS_INSIGHT_URL}/collections"
PERIOD_VALUES = ["past_24_hours", "past_week", "past_month", "past_3_months"]

# Initialize API Clients
openai_client = AsyncOpenAI(api_key=os.getenv("OPENAI_PROJECT_KEY"))
exa_client = Exa(api_key=os.getenv("EXA_API_KEY"))
github_client = Github(os.getenv("GITHUB_TOKEN"))

# LLM Model Configuration
REASONING_MODELS = [
    "o1", "o1-2024-12-17", "o3-mini-2025-01-31", "o3-mini",
    "o4-mini", "o4-mini-2025-04-16", "o3", "o3-2025-04-16"
]

LLM_MODEL_COSTS = {
    "o3": {"input": 2 / 10 ** 6, "cached_input": 0.6 / 10 ** 6, "output": 8 / 10 ** 6},
    "o3-2025-04-16": {"input": 10.0 / 10 ** 6, "cached_input": 2.5 / 10 ** 6, "output": 40.0 / 10 ** 6},
    "o4-mini": {"input": 1.1 / 10 ** 6, "cached_input": 0.275 / 10 ** 6, "output": 4.4 / 10 ** 6},
    "o4-mini-2025-04-16": {"input": 1.1 / 10 ** 6, "cached_input": 0.275 / 10 ** 6, "output": 4.4 / 10 ** 6},
    "gpt-4.1-mini": {"input": 0.40 / 10 ** 6, "cached_input": 0.10 / 10 ** 6, "output": 1.60 / 10 ** 6},
    "gpt-4.1": {"input": 2.00 / 10 ** 6, "cached_input": 0.50 / 10 ** 6, "output": 8.00 / 10 ** 6},
    "gpt-4.1-nano": {"input": 0.10 / 10 ** 6, "cached_input": 0.025 / 10 ** 6, "output": 0.40 / 10 ** 6},
    "gpt-4o": {"input": 2.5 / 10 ** 6, "cached_input": 1.25 / 10 ** 6, "output": 10 / 10 ** 6},
    "gpt-4o-2024-11-20": {"input": 2.5 / 10 ** 6, "cached_input": 1.25 / 10 ** 6, "output": 10 / 10 ** 6},
    "gpt-4o-2024-08-06": {"input": 2.5 / 10 ** 6, "cached_input": 1.25 / 10 ** 6, "output": 10 / 10 ** 6},
    "gpt-4o-mini": {"input": 0.15 / 10 ** 6, "cached_input": 0.075 / 10 ** 6, "output": 0.6 / 10 ** 6},
    "text-embedding-3-small": {"input": 0.02 / 10 ** 6},
    "text-embedding-3-large": {"input": 0.13 / 10 ** 6},
    "text-embedding-ada-002": {"input": 0.1 / 10 ** 6}
}

# Pydantic Models for VC Analysis
class Collection(BaseModel):
    id: str
    name: str

class DevOpsCollectionsResponse(BaseModel):
    collections: List[Collection]

class READMEAnalysis(BaseModel):
    clarity_score: int  # Score from 1-5 on how clear the README is
    problem_solved: str  # A summary of the problem the repo solves
    time_to_wow_estimate: str  # e.g., "Minutes", "Hours", "Days"
    is_painkiller: bool  # Does it solve a common, frustrating problem?

class SentimentAnalysis(BaseModel):
    overall_sentiment: str  # e.g., "Positive", "Neutral", "Mixed", "Negative"
    sentiment_score: int  # Score from 1-5
    key_quotes: List[str]  # Positive or insightful quotes from discussions
    common_criticisms: List[str]  # Common complaints or concerns

class DevOpsClassification(BaseModel):
    is_devops_related: bool  # True if this repository is DevOps/infrastructure related
    confidence_score: int  # Score from 1-5, how confident the classification is
    reasoning: str  # Brief explanation of why it's classified as DevOps or not
    devops_categories: List[str]  # Categories like "CI/CD", "Container", "Monitoring", etc.

class FinalAnalysis(BaseModel):
    repo_name: str
    oss_insight_data: dict
    readme_analysis: Optional[READMEAnalysis] = None
    sentiment_analysis: Optional[SentimentAnalysis] = None
    score: Optional[float] = None  # Calculated final score

print("✅ VC Sourcing Pipeline initialized successfully")
print(f"🔑 API clients configured for OpenAI, Exa, and GitHub")
print(f"📊 Ready to analyze DevOps repositories across {len(PERIOD_VALUES)} time periods")

✅ VC Sourcing Pipeline initialized successfully
🔑 API clients configured for OpenAI, Exa, and GitHub
📊 Ready to analyze DevOps repositories across 4 time periods


In [33]:
# Cost Tracking for LLM Usage
class ModelUsageAsync:
    """Track LLM API costs throughout the VC analysis pipeline."""
    
    def __init__(self, model_costs=LLM_MODEL_COSTS, web_search_costs=None):
        self.token_usage = {}
        self.web_search_usage = {}
        self.model_costs = model_costs
        self.web_search_costs = web_search_costs
        self.logs = []
        self.lock = asyncio.Lock()

    async def add_tokens(
        self, 
        model: str, 
        input_tokens: int = 0, 
        output_tokens: int = 0, 
        cached_tokens: int = 0, 
        reasoning_tokens: int = 0, 
        label: Optional[str] = None
    ) -> None:
        """Add token usage for cost tracking."""
        async with self.lock:
            if model not in self.token_usage:
                self.token_usage[model] = {"input": 0, "output": 0, "cached": 0, "reasoning": 0}
            
            self.token_usage[model]["input"] += input_tokens
            self.token_usage[model]["output"] += output_tokens
            self.token_usage[model]["cached"] += cached_tokens
            self.token_usage[model]["reasoning"] += reasoning_tokens
            
            if label is not None:
                self.logs.append({
                    "model": model, "input_tokens": input_tokens, "output_tokens": output_tokens,
                    "cached_tokens": cached_tokens, "reasoning_tokens": reasoning_tokens, "label": label
                })

    async def add_web_search_usage(self, model: str, search_context_size: str) -> None:
        """Add web search usage for cost tracking."""
        if self.web_search_costs is not None:
            if model not in self.web_search_usage:
                self.web_search_usage[model] = {}
            if search_context_size not in self.web_search_usage[model]:
                self.web_search_usage[model][search_context_size] = 0
            self.web_search_usage[model][search_context_size] += 1
        
    async def get_cost(self) -> float:
        """Calculate total cost of LLM usage."""
        async with self.lock:
            for model in self.token_usage:
                if model not in self.model_costs:
                    raise ValueError(f"Model {model} is not supported.")
            
            cost_of_input_output = sum([
                (self.token_usage[model]["input"] - self.token_usage[model]["cached"]) * self.model_costs[model].get("input", 0) +
                self.token_usage[model]["output"] * self.model_costs[model].get("output", 0) +
                self.token_usage[model]["cached"] * self.model_costs[model].get("cached_input", 0)
                for model in self.token_usage 
            ]) 

            cost_of_web_searches = 0
            if self.web_search_costs is not None:
                for model in self.web_search_usage:
                    for search_context_size in self.web_search_usage[model]:
                        cost_of_web_searches += self.web_search_usage[model][search_context_size] * self.web_search_costs[model].get(search_context_size, 0)

            return cost_of_input_output + cost_of_web_searches

    async def get_tokens_used(self) -> int:
        """Get total number of tokens used."""
        async with self.lock:
            return sum([
                self.token_usage[model]["input"] + self.token_usage[model]["output"]
                for model in self.token_usage
            ])

    async def get_web_searches_performed(self) -> int:
        """Get total number of web searches performed."""
        async with self.lock:
            total_searches = 0
            for model in self.web_search_usage:
                for search_context_size in self.web_search_usage[model]:
                    total_searches += self.web_search_usage[model][search_context_size]
            return total_searches


In [34]:
# OpenAI API Helper with Structured Output
async def call_openai_structured(
    openai_client: AsyncOpenAI,
    model: str,
    llm_usage: Optional[ModelUsageAsync] = None,
    llm_usage_label: Optional[str] = None,
    max_retries: int = 3,
    base_delay: int = 5,
    completion_timeout: int = 90,
    web_search_bool: bool = False,
    web_search_context_size: str = "high",
    **kwargs
) -> Optional[ChatCompletion]:
    """Call OpenAI API with structured output and cost tracking."""
    
    # Handle model-specific parameters
    if model in REASONING_MODELS:
        kwargs.pop("temperature", None)  # Reasoning models don't support temperature
    else:
        kwargs.pop("reasoning_effort", None)  # Only supported by reasoning models
    
    for attempt in range(max_retries):
        try:
            completion = await asyncio.wait_for(
                openai_client.chat.completions.parse(model=model, **kwargs),
                timeout=completion_timeout
            )
            
            # Track usage and costs
            if llm_usage is not None:
                await llm_usage.add_tokens(
                    model=model,
                    input_tokens=completion.usage.prompt_tokens,
                    output_tokens=completion.usage.completion_tokens,
                    cached_tokens=completion.usage.prompt_tokens_details.cached_tokens,
                    reasoning_tokens=completion.usage.completion_tokens_details.reasoning_tokens,
                    label=llm_usage_label
                )
                
                if web_search_bool:
                    await llm_usage.add_web_search_usage(model, web_search_context_size)
            
            return completion
            
        except asyncio.TimeoutError:
            print(f"🔄 API timeout for {llm_usage_label or 'request'}, retrying...")
            await asyncio.sleep(base_delay)
        except Exception as e:
            print(f"❌ API error for {llm_usage_label or 'request'}: {e}")
            if attempt == max_retries - 1:
                break
            await asyncio.sleep(base_delay)

    print(f"❌ Max retries exceeded for {llm_usage_label or 'request'}")
    return None


In [35]:
# OSS Insight API Helper
async def make_oss_insight_request(
    session: aiohttp.ClientSession,
    url: str,
    params: dict
) -> Optional[dict]:
    """Make requests to OSS Insight API with error handling."""
    try:
        async with session.get(url, params=params) as response:
            response.raise_for_status()
            data = await response.json()
            return data
    except Exception as e:
        print(f"❌ OSS Insight API error: {e}")
        return None

In [36]:
# DevOps Collections Identification
GET_DEV_OPS_COLLECTIONS_SYSTEM_PROMPT = """
You are a helpful assistant that identifies DevOps collections from a list of collections.
"""

GET_DEV_OPS_COLLECTIONS_USER_PROMPT = """
Task: Identify the DevOps collections from the list of collections.
Return a structured response with collections related to DevOps, CI/CD, infrastructure, monitoring, deployment, or developer tools.

List of collections:
{context}

Your answer must:
1. Only contain collections from the provided list
2. Return collections in the specified format with both id and name fields
3. Be inclusive of any tools that developers would use for infrastructure, automation, or deployment
"""

async def get_dev_ops_collections(
    openai_client: AsyncOpenAI,
    collections_data: dict,
    llm_usage: ModelUsageAsync
) -> List[Dict[str, str]]:
    """Identify DevOps collections using LLM analysis."""
    collections_rows = collections_data.get("data", {}).get("rows", [])
    all_collections = [{row["id"]: row["name"]} for row in collections_rows]
    
    print(f"🔍 Analyzing {len(all_collections)} collections to identify DevOps-related ones...")
    
    context = str(all_collections)
    messages = [
        {"role": "developer", "content": GET_DEV_OPS_COLLECTIONS_SYSTEM_PROMPT},
        {"role": "user", "content": GET_DEV_OPS_COLLECTIONS_USER_PROMPT.format(context=context)}
    ]

    response = await call_openai_structured(
        openai_client=openai_client,
        model="o4-mini",
        messages=messages,
        response_format=DevOpsCollectionsResponse,
        llm_usage=llm_usage,
        llm_usage_label="dev_ops_collections_identification",
        reasoning_effort="high"
    )

    if not response or not response.choices[0].message.parsed:
        print("❌ Failed to identify DevOps collections")
        return []
    
    parsed_response = response.choices[0].message.parsed
    dev_ops_collections = [{c.id: c.name} for c in parsed_response.collections]
    
    print(f"✅ Identified {len(dev_ops_collections)} DevOps collections")
    return dev_ops_collections


In [37]:
# Fetch Trending Repositories Data
print("🚀 Phase 0: Fetching trending repositories data...")

async with aiohttp.ClientSession() as session:
    period_trending_repos = {}
    for period in PERIOD_VALUES:
        print(f"📅 Fetching trending repos for {period}...")
        trending_data = await make_oss_insight_request(
            session=session,
            url=TRENDING_REPOS_URL,
            params={"period": period}
        )
        if trending_data:
            period_trending_repos[period] = trending_data
            repo_count = len(trending_data.get("data", {}).get("rows", []))
            print(f"✅ Found {repo_count} trending repositories for {period}")
        else:
            print(f"❌ Failed to fetch data for {period}")

total_repos = sum(len(data.get("data", {}).get("rows", [])) for data in period_trending_repos.values())
print(f"\n📊 Total repositories fetched: {total_repos} across {len(period_trending_repos)} periods")

🚀 Phase 0: Fetching trending repositories data...
📅 Fetching trending repos for past_24_hours...
✅ Found 100 trending repositories for past_24_hours
📅 Fetching trending repos for past_week...
✅ Found 100 trending repositories for past_week
📅 Fetching trending repos for past_month...
✅ Found 100 trending repositories for past_month
📅 Fetching trending repos for past_3_months...
✅ Found 100 trending repositories for past_3_months

📊 Total repositories fetched: 400 across 4 periods


In [38]:
# Fetch Collections and Identify DevOps Collections
print("🔍 Fetching and analyzing repository collections...")

# Initialize cost tracking for DevOps collections identification
dev_ops_collections_usage = ModelUsageAsync()

async with aiohttp.ClientSession() as session:
    collections_data = await make_oss_insight_request(
        session=session,
        url=COLLECTIONS_URL,
        params={}
    )

if collections_data:
    total_collections = len(collections_data.get("data", {}).get("rows", []))
    print(f"✅ Found {total_collections} total collections")
    
    # Identify DevOps collections using LLM
    dev_ops_collections = await get_dev_ops_collections(
        openai_client=openai_client,
        collections_data=collections_data,
        llm_usage=dev_ops_collections_usage
    )
    
    if dev_ops_collections:
        print(f"📋 DevOps collections identified:")
        for collection in dev_ops_collections[:10]:  # Show first 10
            for id_key, name in collection.items():
                print(f"  • {name} (ID: {id_key})")
        if len(dev_ops_collections) > 10:
            print(f"  ... and {len(dev_ops_collections) - 10} more")
    
    # Display cost information
    cost = await dev_ops_collections_usage.get_cost()
    tokens = await dev_ops_collections_usage.get_tokens_used()
    print(f"💰 DevOps identification cost: ${cost:.4f} ({tokens:,} tokens)")
else:
    print("❌ Failed to fetch collections data")
    dev_ops_collections = []

🔍 Fetching and analyzing repository collections...
✅ Found 95 total collections
🔍 Analyzing 95 collections to identify DevOps-related ones...
✅ Identified 27 DevOps collections
📋 DevOps collections identified:
  • Static Site Generator (ID: 1)
  • Low Code Development Tool (ID: 10003)
  • Github Alternative (ID: 10009)
  • Text Editor (ID: 10015)
  • Chaos Engineering (ID: 10017)
  • APM Tool (ID: 10018)
  • CICD (ID: 10020)
  • API Gateway (ID: 10021)
  • Distributed File Storage (ID: 10025)
  • Testing Tools (ID: 10027)
  ... and 17 more
💰 DevOps identification cost: $0.0268 (7,077 tokens)


In [39]:
# Data Collection Summary
print("📊 Data Collection Phase Complete")
print("=" * 50)

# Summary of collected data
if 'period_trending_repos' in globals():
    print(f"✅ Trending Repositories: {len(period_trending_repos)} time periods")
    for period, data in period_trending_repos.items():
        if data:
            count = len(data.get("data", {}).get("rows", []))
            print(f"   • {period}: {count} repositories")

if 'dev_ops_collections' in globals():
    print(f"✅ DevOps Collections: {len(dev_ops_collections)} identified")
    
print("\n🎯 Ready to proceed with VC sourcing pipeline!")
print("📋 Data available for analysis:")
print("   • Trending repository data across multiple time periods")
print("   • DevOps-specific collection classifications")
print("   • API clients configured for GitHub, Exa, and OpenAI")

📊 Data Collection Phase Complete
✅ Trending Repositories: 4 time periods
   • past_24_hours: 100 repositories
   • past_week: 100 repositories
   • past_month: 100 repositories
   • past_3_months: 100 repositories
✅ DevOps Collections: 27 identified

🎯 Ready to proceed with VC sourcing pipeline!
📋 Data available for analysis:
   • Trending repository data across multiple time periods
   • DevOps-specific collection classifications
   • API clients configured for GitHub, Exa, and OpenAI


In [40]:
# LLM-Based DevOps Repository Classification
CLASSIFY_DEVOPS_REPO_SYSTEM_PROMPT = """
You are an expert VC analyst specializing in DevOps and infrastructure tooling. Your task is to classify whether a GitHub repository is related to DevOps, infrastructure, or developer productivity tools.
"""

CLASSIFY_DEVOPS_REPO_USER_PROMPT = """
Repository: {repo_name}
Description: {description}

Based on the repository name and description, determine if this is a DevOps-related project.

DevOps categories include:
- CI/CD and deployment pipelines
- Container technologies (Docker, Kubernetes, etc.)
- Infrastructure as Code (Terraform, Ansible, etc.)
- Monitoring and observability tools
- Cloud platforms and services
- Developer productivity tools
- Security and compliance tools
- Database and data infrastructure
- API gateways and service mesh
- Configuration management
- Build and package management tools

Analyze this repository and provide:
1. **is_devops_related**: True if it's DevOps/infrastructure related
2. **confidence_score**: Your confidence (1-5, where 5 is very confident)
3. **reasoning**: Brief explanation of your classification
4. **devops_categories**: List of relevant DevOps categories (empty list if not DevOps)
"""

async def classify_repository_with_llm(
    openai_client: AsyncOpenAI,
    repo_name: str,
    description: str,
    llm_usage: ModelUsageAsync
) -> Optional[DevOpsClassification]:
    """Classify a repository as DevOps-related using LLM analysis."""
    
    messages = [
        {"role": "developer", "content": CLASSIFY_DEVOPS_REPO_SYSTEM_PROMPT},
        {"role": "user", "content": CLASSIFY_DEVOPS_REPO_USER_PROMPT.format(
            repo_name=repo_name,
            description=description or "No description available"
        )}
    ]

    response = await call_openai_structured(
        openai_client=openai_client,
        model="o4-mini",
        messages=messages,
        response_format=DevOpsClassification,
        llm_usage=llm_usage,
        llm_usage_label=f"devops_classification_{repo_name}",
        reasoning_effort="medium"
    )

    if response and response.choices:
        return response.choices[0].message.parsed
    
    return None

print("✅ LLM-based DevOps classification function loaded")


✅ LLM-based DevOps classification function loaded


In [41]:
# Helper Functions for VC Analysis
def truncate_content_for_llm(content: str, max_chars: int = 15000) -> str:
    """Truncate content to fit within LLM context limits."""
    return content[:max_chars] if len(content) > max_chars else content

# Repository Filtering
def filter_devops_repositories(
    period_trending_repos: Dict[str, dict],
    dev_ops_collections: List[Dict[str, str]]
) -> Dict[str, dict]:
    """Filter repositories that match DevOps collections."""
    # Extract collection names from the DevOps collections
    dev_ops_collection_names = {
        name for collection_dict in dev_ops_collections 
        for name in collection_dict.values()
    }
    
    potential_leads = {}
    
    print(f"🔍 Filtering repositories using {len(dev_ops_collection_names)} DevOps collections...")
    
    for period, trending_data in period_trending_repos.items():
        if not trending_data or not trending_data.get("data"):
            continue
            
        print(f"📅 Processing {period}...")
        
        for repo in trending_data["data"]["rows"]:
            repo_collections = set(repo.get("collection_names", "").split(','))
            repo_collections = {c.strip() for c in repo_collections if c.strip()}  # Clean up
            
            # Check for intersection between repo's collections and our DevOps list
            if not dev_ops_collection_names.isdisjoint(repo_collections):
                repo_name = repo["repo_name"]
                if repo_name not in potential_leads:
                    potential_leads[repo_name] = repo
                    matching_collections = repo_collections.intersection(dev_ops_collection_names)
                    print(f"✅ Found DevOps repo: {repo_name} (matches: {matching_collections})")
    
    print(f"\n🎯 Found {len(potential_leads)} potential DevOps leads across all periods.")
    return potential_leads


In [42]:
# README Analysis Function
async def analyze_readme(
    openai_client: AsyncOpenAI,
    github_client: Github,
    repo_name: str,
    llm_usage: ModelUsageAsync
) -> Optional[READMEAnalysis]:
    """Analyze repository README for VC investment potential."""
    try:
        repo = github_client.get_repo(repo_name)
        readme_content = repo.get_readme().decoded_content.decode("utf-8")
        print(f"✅ Fetched README for {repo_name}")
    except Exception as e:
        print(f"❌ Could not fetch README for {repo_name}: {e}")
        return None

    # Truncate README to fit context window
    readme_content = truncate_content_for_llm(readme_content)

    system_prompt = "You are an expert VC analyst. Your task is to analyze a project's README file to assess its investment potential based on clarity and the problem it solves."
    user_prompt = f"""
    Analyze the following README.md content for the repository '{repo_name}'.
    
    README Content:
    ---
    {readme_content}
    ---
    
    Based on the content, please provide the following analysis:
    1. **Clarity Score (1-5):** How clearly and concisely does it state the problem and solution? 5 is extremely clear.
    2. **Problem Solved:** In one sentence, what specific problem does this project solve?
    3. **Time to 'Wow' Estimate:** Based on the 'Getting Started' guide, estimate the time for a developer to get a valuable outcome (e.g., 'Minutes', 'Hours', 'Days').
    4. **Is Painkiller:** Does this project seem to solve an acute, frustrating 'painkiller' problem, or is it a 'vitamin' (nice-to-have)?
    """
    
    response = await call_openai_structured(
        openai_client=openai_client,
        model="o4-mini",
        messages=[{"role": "developer", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        response_format=READMEAnalysis,
        llm_usage=llm_usage,
        llm_usage_label=f"readme_analysis_{repo_name}",
        reasoning_effort="high"
    )

    if response and response.choices:
        print(f"✅ README analysis completed for {repo_name}")
        return response.choices[0].message.parsed
    
    print(f"❌ README analysis failed for {repo_name}")
    return None


In [43]:
# Community Sentiment Analysis Function
async def analyze_community_sentiment(
    openai_client: AsyncOpenAI,
    exa_client: Exa,
    repo_name: str,
    llm_usage: ModelUsageAsync
) -> Optional[SentimentAnalysis]:
    """Analyze community sentiment using Exa search and LLM analysis."""
    # Use Exa to find relevant discussions
    query = f"discussions and reviews of the GitHub repository '{repo_name}'"
    try:
        search_response = exa_client.search_and_contents(
            query,
            num_results=5,
            include_domains=["news.ycombinator.com", "reddit.com"],
            text=True
        )
        
        if not search_response.results:
            print(f"⚠️ No Exa search results found for {repo_name}")
            return None
            
        # Format the context for the LLM
        context = "\n\n".join([
            f"URL: {res.url}\nContent: {res.text}" 
            for res in search_response.results
        ])
        print(f"✅ Found {len(search_response.results)} community discussions for {repo_name}")
        
    except Exception as e:
        print(f"❌ Exa search failed for {repo_name}: {e}")
        return None

    system_prompt = "You are a VC analyst specializing in developer sentiment. Analyze the following search results from Hacker News and Reddit to gauge community perception of a software project."
    user_prompt = f"""
    Project: {repo_name}

    Discussion Threads:
    ---
    {context}
    ---

    Based on these discussions, please provide a sentiment analysis:
    1. **Overall Sentiment:** What is the general feeling (Positive, Neutral, Mixed, Negative)?
    2. **Sentiment Score (1-5):** How excited is the community? 5 is viral excitement.
    3. **Key Quotes:** List up to 3 direct quotes that capture the essence of the community's praise or key insights.
    4. **Common Criticisms:** List up to 2 common criticisms or concerns raised by the community.
    """

    response = await call_openai_structured(
        openai_client=openai_client,
        model="o4-mini",
        messages=[{"role": "developer", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        response_format=SentimentAnalysis,
        llm_usage=llm_usage,
        llm_usage_label=f"sentiment_analysis_{repo_name}",
        reasoning_effort="high"
    )
    
    if response and response.choices:
        print(f"✅ Sentiment analysis completed for {repo_name}")
        return response.choices[0].message.parsed
    
    print(f"❌ Sentiment analysis failed for {repo_name}")
    return None


In [44]:
# Scoring System and Pipeline Orchestration
def calculate_final_score(analysis: FinalAnalysis) -> float:
    """Calculate final investment score based on multiple factors."""
    score = 0.0
    
    # 1. Popularity Score (from OSS Insight total_score)
    total_score = float(analysis.oss_insight_data.get("total_score", 0))
    score += min(total_score / 1000, 5.0)  # Normalize to max 5 points
    
    # 2. README Quality Score
    if analysis.readme_analysis:
        score += analysis.readme_analysis.clarity_score
        if analysis.readme_analysis.is_painkiller:
            score += 2.0  # Bonus for solving painful problems
        
        # Time to wow bonus
        time_to_wow = analysis.readme_analysis.time_to_wow_estimate.lower()
        if "minute" in time_to_wow:
            score += 2.0
        elif "hour" in time_to_wow:
            score += 1.0
    
    # 3. Community Sentiment Score
    if analysis.sentiment_analysis:
        score += analysis.sentiment_analysis.sentiment_score
        
        # Bonus for positive sentiment
        if analysis.sentiment_analysis.overall_sentiment.lower() == "positive":
            score += 1.0
    
    return round(score, 2)

# Repository Analysis Pipeline
async def analyze_repository_pipeline(
    repo_name: str,
    repo_data: dict,
    openai_client: AsyncOpenAI,
    github_client: Github,
    exa_client: Exa,
    llm_usage: ModelUsageAsync
) -> FinalAnalysis:
    """Complete analysis pipeline for a single repository."""
    print(f"🔍 Analyzing {repo_name}...")
    
    # Parallel analysis where possible
    readme_task = analyze_readme(openai_client, github_client, repo_name, llm_usage)
    sentiment_task = analyze_community_sentiment(openai_client, exa_client, repo_name, llm_usage)
    
    readme_analysis, sentiment_analysis = await asyncio.gather(
        readme_task,
        sentiment_task,
        return_exceptions=True
    )
    
    # Handle exceptions
    if isinstance(readme_analysis, Exception):
        print(f"⚠️ README analysis failed for {repo_name}: {readme_analysis}")
        readme_analysis = None
    
    if isinstance(sentiment_analysis, Exception):
        print(f"⚠️ Sentiment analysis failed for {repo_name}: {sentiment_analysis}")
        sentiment_analysis = None
    
    # Create final analysis
    analysis = FinalAnalysis(
        repo_name=repo_name,
        oss_insight_data=repo_data,
        readme_analysis=readme_analysis,
        sentiment_analysis=sentiment_analysis
    )
    
    # Calculate score
    analysis.score = calculate_final_score(analysis)
    
    return analysis


In [45]:
# Reporting Functions
def print_analysis_summary(analysis: FinalAnalysis) -> None:
    """Print formatted analysis summary."""
    print(f"\n## {analysis.repo_name} | 📊 Score: {analysis.score}")
    print(f"📈 Total Score: {analysis.oss_insight_data.get('total_score')} | ⭐ Stars: {analysis.oss_insight_data.get('stars')}")
    print(f"📝 Description: {analysis.oss_insight_data.get('description')}")
    
    if analysis.readme_analysis:
        print(f"📖 Problem Solved: {analysis.readme_analysis.problem_solved}")
        print(f"🎯 Clarity Score: {analysis.readme_analysis.clarity_score}/5")
        print(f"⚡ Time to Wow: {analysis.readme_analysis.time_to_wow_estimate}")
        print(f"💊 Is Painkiller: {'Yes' if analysis.readme_analysis.is_painkiller else 'No'}")
    
    if analysis.sentiment_analysis:
        print(f"😊 Community Sentiment: {analysis.sentiment_analysis.overall_sentiment}")
        print(f"📈 Sentiment Score: {analysis.sentiment_analysis.sentiment_score}/5")
        if analysis.sentiment_analysis.key_quotes:
            print(f"💬 Key Quotes: {analysis.sentiment_analysis.key_quotes}")
        if analysis.sentiment_analysis.common_criticisms:
            print(f"⚠️ Common Criticisms: {analysis.sentiment_analysis.common_criticisms}")

def print_final_report(final_results: List[FinalAnalysis], total_usage: ModelUsageAsync) -> None:
    """Print the final investment report."""
    # Sort results by score
    sorted_results = sorted(final_results, key=lambda x: x.score or 0, reverse=True)
    
    print("\n\n" + "="*80)
    print("📈 FINAL VC INVESTMENT REPORT")
    print("="*80)
    
    for i, result in enumerate(sorted_results, 1):
        print(f"\n🏆 RANK #{i}")
        print_analysis_summary(result)
    
    print("\n" + "="*80)
    print("📊 PIPELINE STATISTICS")
    print("="*80)
    print(f"🔢 Total Repositories Analyzed: {len(final_results)}")
    print(f"✅ Successful README Analyses: {sum(1 for r in final_results if r.readme_analysis)}")
    print(f"😊 Successful Sentiment Analyses: {sum(1 for r in final_results if r.sentiment_analysis)}")
    print(f"💰 Analysis Completeness: {len([r for r in final_results if r.readme_analysis and r.sentiment_analysis])}/{len(final_results)} repositories fully analyzed")


In [49]:
# Production-Ready 3-Tier Filtering (Type-Safe)
async def filter_devops_repositories_production(
    period_trending_repos: Dict[str, dict],
    dev_ops_collections: List[Dict[str, str]],
    openai_client: AsyncOpenAI,
    use_llm_classification: bool = True,
    fallback_to_keywords: bool = True,
    llm_batch_size: int = 10
) -> Dict[str, dict]:
    """Production-ready 3-tier filtering with proper error handling."""
    
    # TIER 1: Collection-based filtering
    dev_ops_collection_names = {
        name for collection_dict in dev_ops_collections 
        for name in collection_dict.values()
    }
    
    potential_leads = {}
    collection_matches = 0
    llm_matches = 0
    keyword_matches = 0
    
    # DevOps keywords for final fallback
    devops_keywords = {
        'docker', 'kubernetes', 'k8s', 'terraform', 'ansible', 'jenkins', 
        'ci/cd', 'cicd', 'deployment', 'infrastructure', 'devops', 'monitoring',
        'prometheus', 'grafana', 'elasticsearch', 'nginx', 'apache', 'redis',
        'postgresql', 'mysql', 'microservices', 'container', 'orchestration',
        'automation', 'pipeline', 'cloud', 'aws', 'azure', 'gcp', 'helm'
    }
    
    print(f"🔍 3-Tier Filtering Strategy: Collections → LLM → Keywords")
    print(f"📋 Using {len(dev_ops_collection_names)} DevOps collections for Tier 1...")
    
    # Collect repositories for processing
    unmatched_repos = []
    
    for period, trending_data in period_trending_repos.items():
        if not trending_data or not trending_data.get("data"):
            continue
            
        print(f"📅 Processing {period}...")
        
        for repo in trending_data["data"]["rows"]:
            repo_name = repo["repo_name"]
            repo_collections = set(repo.get("collection_names", "").split(','))
            repo_collections = {c.strip() for c in repo_collections if c.strip()}
            
            # TIER 1: Collection-based filtering
            if not dev_ops_collection_names.isdisjoint(repo_collections):
                if repo_name not in potential_leads:
                    potential_leads[repo_name] = repo
                    matching_collections = repo_collections.intersection(dev_ops_collection_names)
                    print(f"✅ Tier 1 - Collection match: {repo_name}")
                    collection_matches += 1
                    continue
            
            # Store for LLM analysis
            if repo_name not in potential_leads:
                unmatched_repos.append(repo)
    
        # TIER 2: LLM-based classification (PARALLEL for speed)
        if use_llm_classification and unmatched_repos:
            print(f"\n🤖 Tier 2 - LLM Classification on {len(unmatched_repos)} repositories...")
            llm_usage = ModelUsageAsync()
            
            # Process in parallel batches for speed
            total_batches = (len(unmatched_repos) + llm_batch_size - 1) // llm_batch_size
            
            for batch_idx in range(0, len(unmatched_repos), llm_batch_size):
                batch = unmatched_repos[batch_idx:batch_idx + llm_batch_size]
                batch_num = (batch_idx // llm_batch_size) + 1
                
                print(f"🔄 Processing batch {batch_num}/{total_batches} ({len(batch)} repositories)")
                
                # Create parallel classification tasks
                classification_tasks = [
                    classify_repository_with_llm(
                        openai_client=openai_client,
                        repo_name=repo["repo_name"],
                        description=repo.get("description", ""),
                        llm_usage=llm_usage
                    )
                    for repo in batch if repo["repo_name"] not in potential_leads
                ]
                
                # Execute batch in parallel
                try:
                    classifications = await asyncio.gather(*classification_tasks, return_exceptions=True)
                    
                    # Process results with explicit type handling
                    repos_in_batch = [repo for repo in batch if repo["repo_name"] not in potential_leads]
                    
                    for repo, result in zip(repos_in_batch, classifications):
                        # Handle exceptions explicitly
                        if isinstance(result, Exception):
                            print(f"⚠️ LLM classification failed for {repo['repo_name']}: {result}")
                            continue
                        
                        # Type-safe processing: result is now Optional[DevOpsClassification]
                        if result is not None:
                            try:
                                if result.is_devops_related:
                                    repo_name = repo["repo_name"]
                                    potential_leads[repo_name] = repo
                                    categories_str = ', '.join(result.devops_categories[:2]) if result.devops_categories else 'General DevOps'
                                    print(f"🧠 Tier 2 - LLM match: {repo_name} (confidence: {result.confidence_score}/5)")
                                    llm_matches += 1
                            except AttributeError:
                                # Additional safety for any attribute access issues
                                print(f"⚠️ Invalid classification result for {repo['repo_name']}")
                                
                except Exception as e:
                    print(f"⚠️ Batch processing failed: {e}")
            
            # Show LLM costs
            llm_cost = await llm_usage.get_cost()
            print(f"💰 LLM classification cost: ${llm_cost:.4f}")
    
    # TIER 3: Keyword-based final fallback
    if fallback_to_keywords:
        print(f"\n🔍 Tier 3 - Keyword-based fallback...")
        for repo in unmatched_repos:
            repo_name = repo["repo_name"]
            if repo_name in potential_leads:
                continue  # Already matched
                
            description = repo.get("description", "").lower()
            repo_name_lower = repo_name.lower()
            
            # Check for DevOps keywords
            found_keywords = []
            for keyword in devops_keywords:
                if keyword in description or keyword in repo_name_lower:
                    found_keywords.append(keyword)
            
            if found_keywords:
                potential_leads[repo_name] = repo
                print(f"🎯 Tier 3 - Keyword match: {repo_name}")
                keyword_matches += 1
    
    print(f"\n📊 3-Tier Filtering Results:")
    print(f"   • Tier 1 (Collections): {collection_matches}")
    print(f"   • Tier 2 (LLM): {llm_matches}")
    print(f"   • Tier 3 (Keywords): {keyword_matches}")
    print(f"   • Total unique leads: {len(potential_leads)}")
    
    return potential_leads

print("✅ Production-ready 3-tier filtering function loaded")


✅ Production-ready 3-tier filtering function loaded


In [ ]:
# Main VC Sourcing Pipeline Execution with 3-Tier Filtering
async def run_vc_sourcing_pipeline(use_3tier_filtering: bool = True):
    """Execute the complete VC sourcing pipeline with enhanced 3-tier filtering."""
    print("🚀 Starting VC Sourcing Pipeline...")
    
    # Phase 1: Filter DevOps repositories from trending data
    if 'period_trending_repos' not in globals() or 'dev_ops_collections' not in globals():
        print("❌ Missing required data. Please run the previous cells first.")
        return

    # TODO: for demo, just use the repos in past month
    past_month_repos = period_trending_repos["past_month"]
    test_repos = {
        "past_month": past_month_repos
    }
    
    if use_3tier_filtering:
        print("🔍 Using 3-Tier Enhanced Filtering: Collections → LLM → Keywords")
        potential_leads = await filter_devops_repositories_production(
            period_trending_repos=test_repos,
            dev_ops_collections=dev_ops_collections,
            openai_client=openai_client,
            use_llm_classification=True,
            fallback_to_keywords=True,
            llm_batch_size=10
        )
    else:
        print("🔍 Using Original Collection-Only Filtering")
        potential_leads = filter_devops_repositories(test_repos, dev_ops_collections)
    
    if not potential_leads:
        print("❌ No DevOps repositories found. Pipeline ended.")
        return
    
    # Initialize usage tracker for the entire analysis run
    total_usage = ModelUsageAsync()
    final_results = []
    
    # TODO: for demo, just analyze first 5 repos instead of the entirety of potential_leads
    print(f"\n🔬 Phase 2 & 3: Analyzing 5 repositories...")
    
    # Limit analysis to first 5 repositories for demonstration
    # Remove this limit for full analysis
    limited_leads = dict(list(potential_leads.items())[:5])
    print(f"📝 Note: Analyzing first {len(limited_leads)} repositories for demonstration")
    
    # Analyze each repository
    for repo_name, repo_data in limited_leads.items():
        try:
            analysis_result = await analyze_repository_pipeline(
                repo_name=repo_name,
                repo_data=repo_data,
                openai_client=openai_client,
                github_client=github_client,
                exa_client=exa_client,
                llm_usage=total_usage
            )
            final_results.append(analysis_result)
            print(f"✅ Completed analysis for {repo_name}")
            
        except Exception as e:
            print(f"❌ Failed to analyze {repo_name}: {e}")
    
    # Phase 4: Generate final report
    print(f"\n🏁 Phase 4: Scoring and Reporting...")
    print_final_report(final_results, total_usage)
    
    # Display cost information
    total_cost = await total_usage.get_cost()
    total_tokens = await total_usage.get_tokens_used()
    print(f"\n💸 Total Analysis Cost: ${total_cost:.4f}")
    print(f"🔢 Total Tokens Used: {total_tokens:,}")
    
    return final_results

print("✅ VC Sourcing Pipeline functions loaded")
print("📋 Ready to execute pipeline - run the next cell to start analysis")


✅ VC Sourcing Pipeline functions loaded
📋 Ready to execute pipeline - run the next cell to start analysis


In [ ]:
# Execute with 3-Tier Enhanced Filtering
await run_vc_sourcing_pipeline(use_3tier_filtering=True)

🚀 Starting VC Sourcing Pipeline...
🔍 Using 3-Tier Enhanced Filtering: Collections → LLM → Keywords
🔍 3-Tier Filtering Strategy: Collections → LLM → Keywords
📋 Using 27 DevOps collections for Tier 1...
📅 Processing past_month...
✅ Tier 1 - Collection match: n8n-io/n8n

🤖 Tier 2 - LLM Classification on 99 repositories...
🔄 Processing batch 1/10 (10 repositories)
🔄 Processing batch 2/10 (10 repositories)
🧠 Tier 2 - LLM match: anthropics/claude-code (confidence: 4/5)
🧠 Tier 2 - LLM match: sst/opencode (confidence: 3/5)
🧠 Tier 2 - LLM match: bytedance/trae-agent (confidence: 4/5)
🔄 Processing batch 3/10 (10 repositories)
🧠 Tier 2 - LLM match: getAsterisk/claudia (confidence: 3/5)
🧠 Tier 2 - LLM match: ripienaar/free-for-dev (confidence: 5/5)
🧠 Tier 2 - LLM match: microsoft/vscode-copilot-chat (confidence: 5/5)
🧠 Tier 2 - LLM match: upstash/context7 (confidence: 2/5)
🧠 Tier 2 - LLM match: NirDiamant/agents-towards-production (confidence: 3/5)
🔄 Processing batch 4/10 (10 repositories)
🧠 Tier 